# This notebook contains operations related to IMS

In [1]:
import json
import pandas as pd
from IPython.display import display

In [2]:
# loading all the data into notebook
fd=open('records.json',"r")
txt=fd.read()
fd.close()

In [3]:
#converting txt from str to dict.
print(f'before loads fun:{type(txt)}')
records = json.loads(txt)
print(f'after loads fun:{type(records)}')

before loads fun:<class 'str'>
after loads fun:<class 'dict'>


In [4]:
# loading store employees record.
fe=open('employee.json',"r")
emp=fe.read()
fe.close()
employees = json.loads(emp)

In [5]:
employees

{'22345': 'suresh', '22350': 'deepti', '22355': 'rahul'}

## Main operation function.

In [6]:
def ims_operation():
    k=int(input("select:\n1:if you are customer\n2: if you are employee at this store:"))
    if(k==1):
        customer()
    elif(k==2):
        eid=input("enter the employee id:")
        if eid in employees.keys():
            print(f"employee name:{employees[eid]}")
            employee()
        else:
            print("employee id is not registered.")
    else:
        print("not a valid option")


## customer functionalities

In [7]:
def customer():
    choice=int(input("""enter:
                            1: for checking all the products into the store
                            2: for Auto removing the product(recommended before purchasing)
                            3: for Purchasing the product
                            4: for checking trending product
                     """))
    if(choice==1):
        df1=all_products()
        display(df1)
    elif(choice==2):
        auto_update()
    elif(choice==3):
        purchase_item()
    elif(choice==4):
        p_trending()
    else:
        print("not a valid choice")
        

## Employee functionalities

In [8]:
def employee():
    choice=int(input(""" enter:
                             1: for adding the product
                             2: for manually removing the product
                             3: for auto removing the product (when product goes out of stock)
                             4: for checking the trending product
                             5: for updating the product manually
                             6: for checking the sales history
                             7: for updating the employee record
                      """))
    if(choice==1):
        p_adder()
    elif(choice==2):
        p_remover()
    elif(choice==3):
        auto_update()
    elif(choice==4):
        p_trending()
    elif(choice==5):
        P_update_m()
    elif(choice==6):
        df2=sales_history()
        display(df2)
    elif(choice==7):
        employee_update()
    else:
        print("not a valid choice")

## Function for purchasing and updation after generation

In [9]:
import pandas as pd
def purchase_item():
    name= input("please enter your name:")
    ct=0 # overall cost
    while(1):
        pid = input("please scan the bar code or enter the product id:")
        df=pd.DataFrame.from_dict(records[pid],orient='index')
        print("_______________________________________________")
        print(f'product details:{df}')
        print("_______________________________________________")
        item = int(input("please enter no of items:"))
        if(item<=records[str(pid)]['quantity']):
              cost=records[pid]["ppq"]*item
              ct+=cost
              slip_generation(item,cost)
              records[str(pid)]['quantity']-=item
              history_generator(pid,item,cost,name)
              record_update()
        else:
              print(f"there are only {records[pid]['quantity']} items in store")
        i=int(input("If want to purchase more press 1 else any key:"))
        if(i!=1):
            print(f'overall cost:{ct}')
            break
    

## Purchase slip generation

In [10]:
def slip_generation(q,p):
    print("**************************************")
    print(f'No. of quantity purchased:{q}')
    print(f'cost:{p}')
    print("**************************************")

## Purchase history generator

In [11]:
import time
import random
import os
def history_generator(pid,item,cost,name):
    trans_id=int(pid)+records[pid]['quantity']+random.choice(range(150))
    
    fl=open('sales.json',"r+")
    
    if(os.stat('sales.json').st_size==0):
        sales={trans_id:{'customer_name':name,'pname':records[pid]["pname"],'pcategory':records[pid]["pcategory"],
                     'items purchased':item,'amount paid':cost,'time':time.ctime()}}
    else:
        tx=fl.read()
        sales=json.loads(tx)
        sales[trans_id]={'customer_name':name,'pname':records[pid]["pname"],'pcategory':records[pid]["pcategory"],
                         'items purchased':item,'amount paid':cost,'time':time.ctime()}
        fl.truncate(0)  # for deleting all the content
        fl.seek(0)      # for bringing the cursor to 0th index
    txt=json.dumps(sales)
    fl.write(txt)
    fl.close()

## For displaying all the products

In [12]:
def all_products():
    df=pd.DataFrame.from_dict(records,orient='index')
    return df

## Product auto remover
when product goes out of stock

In [13]:
def auto_update():
    keys=records.keys()
    for i in keys:
        if(records[i]['quantity']==0):
            records.pop(i)
    record_update()

## product remover (manually)

In [14]:
def p_remover():
    pid=input("enter the pid of the product which is has to be removed:")
    records.pop(pid)
    record_update()

## product adder

In [15]:
def p_adder():
    pid=input("enter the pid of the product to be added:")
    pname=input("enter the name of the product:")
    pcategory=input("enter the category of the product:")
    psize=input("enter the size of the product:")
    label=input("enter the label of the product:")
    quantity=int(input("enter the no. of products:"))
    ppq=int(input("enter the price per product in rs:"))
    records[pid]={"pname":pname,"pcategory":pcategory,"psize":psize,"label":label,"quantity":quantity,"ppq":ppq}
    record_update()

## Trending product of the day

In [16]:
def p_trending():
    fd=open("sales.json","r")
    txt=fd.read()
    fd.close()
    sales=json.loads(txt)
    k=sales.keys()
    max1=0
    for i in k:
        if(max1<sales[i]['items purchased']):
            max1=sales[i]['items purchased']
    for j in k:
        if(sales[j]['items purchased']==max1):
            print("trending product/products of the day:")
            print(f"product name:{sales[j]['pname']}")
            print(f"product category:{sales[i]['pcategory']}")
            print(f"no. of product purchased:{max1}")
            print("____________________________________________")
        
    
    

## For generating the sales history

In [17]:
def sales_history():
    fd=open("sales.json","r")
    txts=fd.read()
    fd.close()
    r=json.loads(txts)
    dfs=pd.DataFrame.from_dict(r,orient='index')
    return dfs

## Main global file updation

In [18]:
def record_update():
    fd=open('records.json','w')
    txt=json.dumps(records)
    fd.write(txt)
    fd.close

## Product updation (manually)
### In case price changes or quantity increases or decreases
only for store employee

In [19]:
def P_update_m():
    pid=input("enter the id of the product to be updated")
    i=int(input("enter 1 for price and 2 for quantity updation:"))
    if(i==1):
        new_price=int(input("please enter the new price of product"))
        records[pid]['price']=new_price
    elif(i==2):
        new_q=int(input("please enter the new price of product"))
        records[pid]['quantity']=new_q
    else:
        print("invalid option")

## updating employees record
adding or deleting

In [20]:
def employee_update():
    ch=int(input("enter:\n1: for adding\n2: for deleting:"))
    eid=input("enter the id of employee:")
    if(ch==1):
        ename=input("enter the name of employee:")
        employees[eid]=ename
    elif(ch==2):
        employees.pop(eid)
    else:
        print("not a valid choice")
    fe=open('employee.json','w')
    rec=json.dumps(employees)
    fe.write(rec)
    fe.close()
    
        
        

# Main Terminal Window

In [21]:
print("welcome to NI grossary store")
print("please read the instructions and then proceed further according to given options")
print("""
pname: product name
pcategory: product category
ppq:price per quantity (in rs)
quantity: no. of products available
psize: size of the product i.e., in ml or grm
label:certification organization.(NA means no valid certification)
      """)
while(1):
    ims_operation()
    j=int(input("press:\n1: if you want to continue the terminal window,else,\nany key to quit:"))
    if(j!=1):
        break


welcome to NI grossary store
please read the instructions and then proceed further according to given options

pname: product name
pcategory: product category
ppq:price per quantity (in rs)
quantity: no. of products available
psize: size of the product i.e., in ml or grm
label:certification organization.(NA means no valid certification)
      
select:
1:if you are customer
2: if you are employee at this store:2
enter the employee id:22345
employee name:suresh
 enter:
                             1: for adding the product
                             2: for manually removing the product
                             3: for auto removing the product (when product goes out of stock)
                             4: for checking the trending product
                             5: for updating the product manually
                             6: for checking the sales history
                             7: for updating the employee record
                      6


,customer_name,pname,pcategory,items purchased,amount paid,time
10256,anjli,Dove,shampoo,2,120,Sun Sep 5 23:59:03 2021
10233,isha,jasmine,hair oil,1,60,Mon Sep 6 00:07:38 2021
10155,anjali,godrage no 1,human soap cleaner,2,30,Mon Sep 6 00:08:02 2021
10118,Nirbhay,Badshah,namkeen,2,200,Mon Sep 6 12:37:22 2021
10204,Nirbhay,jasmine,hair oil,2,120,Mon Sep 6 12:37:43 2021


press:
1: if you want to continue the terminal window,else,
any key to quit:1
select:
1:if you are customer
2: if you are employee at this store:2
enter the employee id:22345
employee name:suresh
 enter:
                             1: for adding the product
                             2: for manually removing the product
                             3: for auto removing the product (when product goes out of stock)
                             4: for checking the trending product
                             5: for updating the product manually
                             6: for checking the sales history
                             7: for updating the employee record
                      7
enter:
1: for adding
2: for deleting:1
enter the id of employee:22367
enter the name of employee:Kapil
press:
1: if you want to continue the terminal window,else,
any key to quit:1
select:
1:if you are customer
2: if you are employee at this store:1
enter:
                            1: for chec

,pname,pcategory,psize,label,quantity,ppq
10001,Detol handwash,human cleaner,90ml,GMP,100,60
10002,BG handwash,human cleaner,90ml,GMP,80,40
10003,mountain dew,soft drink,250ml,fssai,50,30
10004,sprite,soft drink,500ml,fssai,30,45
10005,pepsi,soft drink,300ml,fssai,60,30
10006,kitkat,chocklate,50gm,fssai,100,20
10007,Dairy milk,chocklate,30gm,fssai,80,15
10008,5 star,chocklate,30gm,fssai,100,10
10009,chokoflex,food substitute,50gm,fssai,50,50
10010,20-20,biskit,60gm,fssai,65,5


press:
1: if you want to continue the terminal window,else,
any key to quit:1
select:
1:if you are customer
2: if you are employee at this store:1
enter:
                            1: for checking all the products into the store
                            2: for Auto removing the product(recommended before purchasing)
                            3: for Purchasing the product
                            4: for checking trending product
                     3
please enter your name:Abhishek
please scan the bar code or enter the product id:10017
_______________________________________________
product details:                             0
pname                 close up
pcategory  human tooth cleaner
psize                     60gm
label                      GMP
quantity                    67
ppq                         45
_______________________________________________
please enter no of items:2
**************************************
No. of quantity purchased:2
cost:90
******************

,customer_name,pname,pcategory,items purchased,amount paid,time
10256,anjli,Dove,shampoo,2,120,Sun Sep 5 23:59:03 2021
10233,isha,jasmine,hair oil,1,60,Mon Sep 6 00:07:38 2021
10155,anjali,godrage no 1,human soap cleaner,2,30,Mon Sep 6 00:08:02 2021
10118,Nirbhay,Badshah,namkeen,2,200,Mon Sep 6 12:37:22 2021
10204,Nirbhay,jasmine,hair oil,2,120,Mon Sep 6 12:37:43 2021
10215,Abhishek,close up,human tooth cleaner,2,90,Mon Sep 6 12:52:05 2021
10325,Abhishek,fair & handsome,M face cream,5,600,Mon Sep 6 12:52:24 2021


press:
1: if you want to continue the terminal window,else,
any key to quit:0
